# Carga de datos

## Librerias

In [ ]:
from os import path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import open
import random

# Importamos las funciones
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
from nltk import pos_tag

# Importaciones del Fetis
import pandas as pd #manejo de datos.
from nltk.probability import FreqDist #gráficos de frecuencia.
import matplotlib.pyplot as plt #mostrar figuras.
from wordcloud import WordCloud #nube de palabras.
from dateutil.parser import parse #analizar strings de fechas.
import numpy as np #manejo numérico.
from nltk.tokenize import RegexpTokenizer #separación por expresiones regulares.
from nltk.corpus import stopwords #lista de stopwords.
from nltk.stem.porter import PorterStemmer #obtener raíz de una palabra.
from sklearn.feature_extraction.text import CountVectorizer #contador de palabras.
from sklearn.model_selection import train_test_split #partición dataset.
from sklearn.linear_model import LogisticRegression #regresión logística.
from sklearn.metrics import accuracy_score #evaluación del clasificador.
from sklearn.metrics import classification_report #evaluación del clasificador.
from sklearn.naive_bayes import GaussianNB # Naive bayes
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

## Constantes

In [ ]:
SAVE_DATAFRAME = False
SAVE_FIG = False
PROCESAR = False
SEED = 42
N_SAMPLES = 2500

# Variables de los nombres de las columnas
TITLE = 'title'
TEXT = 'text'
SUBJECT = 'subject'
DATE = 'date'
TRUE = 'true'

PATH_MAIN = path.join("..")
PATH_DATA = path.join(PATH_MAIN, "datos")
PATH_IMG = path.join(PATH_MAIN, "imagenes")
PATH_LATEX = path.join(PATH_MAIN, "latex")

PATH_TABLAS = path.join(PATH_LATEX, "tablas")

PATH_FAKE = path.join(PATH_DATA, "Fake.csv")
PATH_FAKE_PROS = path.join(PATH_DATA, "Fake_procesado.csv")

PATH_TRUE = path.join(PATH_DATA, "True.csv")
PATH_TRUE_PROS = path.join(PATH_DATA, "True_procesado.csv")

## Carga del dataframe

En esta sección cargamos los DataFrames y además le ponemos una etiqueta de que si es una noticia verdadera o falsa, para posteriormente unir los DataFrames.

### DataFrame de True

In [ ]:
# Nombre de la columna que almacenará la 
# información de si es una noticia verdadera o falsa
name_col = "true"

In [ ]:
# Cargamos DataFrame
df_true = pd.read_csv(PATH_TRUE)

# Agregamos nueva columna
df_true[name_col] = 1

df_true.head()

### DataFrame de Fake

In [ ]:
# Cargamos DataFrame
df_fake = pd.read_csv(PATH_FAKE)

# Agregamos nueva columna
df_fake[name_col] = 0

del name_col

df_fake.head()

### Merge entre DataFrames
Unimos los DataFrames, ahora que se tiene una indexización.

In [ ]:
df = df_true.append(df_fake, ignore_index=True)
df.head()

### Columnas del DataFrame

In [ ]:
# Solo basta un dataframe pues comparten el nombre de las columnas
columnas = list(df.columns)
columnas

## Información de los datasets

### Información de los datasets

Visualizamos la información de las filas de los datasets.

In [ ]:
df_true.info(verbose=True)

In [ ]:
df_fake.info(verbose=True)

In [ ]:
df.info(verbose=True)

### Eliminar los datos que no sean fechas
En algunas filas de los DataFrames hay información etiquetada como fecha, cuando no corresponde a una fecha. Reemplazamos estos valores por un tipo de dato NaT y los dropeamos.

In [ ]:
def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :Param string: str, string to check for date
    :Param fuzzy: bool, ignore unknown tokens in string if True
    """
    from dateutil.parser import parse
    
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [ ]:
nat = np.datetime64('NaT')

def nat_conversor(x): return pd.Timestamp(x) if is_date(x) else nat

df_fake["date"] = df_fake["date"].apply(nat_conversor)
df_true["date"] = df_true["date"].apply(nat_conversor)
df["date"] = df["date"].apply(nat_conversor)

L_df = [df_fake, df_true, df]

In [ ]:
for df_ in L_df:
    df_.dropna(inplace=True)

del nat_conversor, nat

### Variables únicas

En principio, la columna `'title'` y `'text'` no son ni númericas, ni categóricas. `'date'` es un tipo de variable temporal, mientras que `'subject'` es una variable categórica. 

Veamos cuantas variables categóricas diferentes hay para cada `DataFrame`.

In [ ]:
print("El dataset de noticias verdaderas posee las siguientes etiquetas:")
for label in pd.unique(df_true.subject):
    print(f"* {label}")
    
print("\nEl dataset de noticias falsas posee las siguientes etiquetas:")
for label in pd.unique(df_fake.subject):
    print(f"* {label}")

In [ ]:
def imprimir_cantidad_por_categoria(df_, categoria):
    cantidad = len(df_[df_.subject == categoria])
    print(f"La categoría {categoria} posee {cantidad} datos.")
    return None

print("Cantidad de datos por categoría para las noticias verdaderas:")
for label in pd.unique(df_true.subject):
    imprimir_cantidad_por_categoria(df_true, label)

    
print("\nCantidad de datos por categoría para las noticias falsas:")
for label in pd.unique(df_fake.subject):
    imprimir_cantidad_por_categoria(df_fake, label)

### Revisar noticias

¿A qué se refiere `'worldnews'`? Obtengamos una noticia de esta categoría para observarlo:

In [ ]:
def revisar_noticias(df, category, gap=5):
    """Entrega varias noticias de una categoría en específico.
    'df' es el DataFrame, 'category' es la categoría que se quiere revisar
    y 'gap' es el número de noticias que se quiere mostrar.
    """
    df_ = df[df.subject == category]
    df_.reset_index(inplace=True)
    for i in range(gap):
        print("Título:", f'"{df_.title[i]}"')
        print("Texto:", f"{df_.text[i]}")
        print("\n", "-" * 80, "\n")

In [ ]:
revisar_noticias(df_true, "worldnews")

Podemos decir entonces que una noticia de `'worldnews'` es una noticia estadounidense, que se refiere c/r al resto del mundo mundo.

¿Existirá un equivalente en `fakenews`? Empecemos por revisar la categoría `News`.

In [ ]:
revisar_noticias(df_fake, "News")

Parecen ser simplemente noticias estándar, nada relacionado con noticias del mundo.

In [ ]:
revisar_noticias(df_fake, "politics")



Revisemos con la categoría `Middle-east`

In [ ]:
revisar_noticias(df_fake, "Middle-east")

No parece tener relación con noticias del mundo. Veamos ahora la categoría `left-news`

In [ ]:
revisar_noticias(df_fake, 'left-news')

##### Conclusión de Panchito

Con la última categoría tentativa a ser una noticia acerca del mundo, podemos concluir a priori que no hay noticias falsas acerca del resto del mundo. Con lo que una alternativa tentativa para que el algoritmo no sufra de overfitting, es el de eliminar esta categoría.

El problema que tendríamos con esto, es quedarnos únicamente con noticias acerca de la política, así que una opción a esto sería eliminar varias categorías de las noticias falsas, o bien, mantenerlas y decidir cuál específicamente son de política.

Otra opción es predecir sin ocupar esta columna, para no tener ese "spolier" de ser una noticia falsa.

## Procesamiento Preliminar de los datos

In [ ]:
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
lem = WordNetLemmatizer()
stem = PorterStemmer()
stop_words.add('thi')

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
# Obtener una submuestra de los dataframes originales
df_true_ = df_true.copy()
df_fake_ = df_fake.copy()

df_true = df_true.sample(n=N_SAMPLES, random_state=SEED)
df_fake = df_fake.sample(n=N_SAMPLES, random_state=SEED)

### Procesamiento de las noticias falsas

In [ ]:
def procesar(df_a_proc, name_col,
             verbose=True, msg_inicial=None, msg_final=None):
    if verbose: print(msg_inicial)
    col = df_a_proc[name_col]
    for i, text in zip(col.index, col):
        # Dejamos el texto en minusculas
        text = text.lower()
        # Tokenizamos
        text_token = tokenizer.tokenize(text)
        # Procesamos: stemmeamos las palabras que no sean stopwords, y unimos
        text_proc = " ".join([
            lem.lemmatize(word, 
                          get_wordnet_pos(word)
                         ) for word in text_token if word not in stop_words
        ])
        # Reemplazamos el texto procesado en el df a guardar
        df_a_proc[name_col][i] = text_proc
    if verbose: print(msg_final)
    return df_a_proc

In [ ]:
# Titulares noticias falsas transformados
if PROCESAR:
    df_fake_proc = df_fake.copy()
    
    # Procesar el titulo
    procesar(df_fake_proc, TITLE,
             msg_inicial="Procesando los titulares falsos...",
             msg_final="Titulares falsos procesados!")
    
    # Procesar el texto
    procesar(df_fake_proc, TEXT,
             msg_inicial="Procesando los textos falsos...",
             msg_final="Textos falsos procesados!")
    
    if SAVE_DATAFRAME:
        print("Guardando DataFrame procesado...")
        df_fake_proc.dropna(inplace=True)
        df_fake_proc.to_csv(PATH_FAKE_PROS, index=False)
        print("DataFrame procesado guardado!")

else:
    print("Leyendo noticias falsas...")
    df_fake_proc = pd.read_csv(PATH_FAKE_PROS)
    df_fake_proc.dropna(inplace=True)
    print("Noticias falsas leidas!")
    
df_fake_proc.head()

### Prcesamiento de las noticias verdaderas

In [ ]:
# titulares noticias verdaderas transformados
if PROCESAR:
    df_true_proc = df_true.copy()
    
    # Procesar el titulo
    procesar(df_true_proc, TITLE,
             msg_inicial="Procesando los titulares verdaderos...",
             msg_final="Titulares verdaderos procesados!")
    
    # Procesar el texto
    procesar(df_true_proc, TEXT,
             msg_inicial="Procesando los textos verdaderos...",
             msg_final="Textos verdaderos procesados!")
    
    if SAVE_DATAFRAME:
        print("Guardando DataFrame procesado...")
        df_true_proc.dropna(inplace=True)
        df_true_proc.to_csv(PATH_TRUE_PROS, index=False)
        print("DataFrame procesado guardado!")

else:
    print("Leyendo noticias verdaderas...")
    df_true_proc = pd.read_csv(PATH_TRUE_PROS)
    df_true_proc.dropna(inplace=True)
    print("Noticias verdaderas leidas!")
    
df_true_proc.head()

### DataFrame procesado


In [ ]:
df_proc = df_true_proc.append(df_fake_proc, ignore_index=True)
df_proc_ = df_proc
df_proc.head()

# EDA

## Análisis gráfico del tiempo

### Funciones preliminares

In [ ]:
def rotular(ax, title, xlabel, ylabel):
    """Rotula un gráfico
    """
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

In [ ]:
def my_plot(ts_t, ts_f, 
            title=None, xlabel=None, ylabel=None, rotation=40,
            save=SAVE_FIG, save_name=None,
           ):
    """Plotea las cosas bajo el contexto que necesitamos.
    """
    fig, ax = plt.subplots()  # Create a figure containing a single axes.
    ax.plot(ts_f)  # Plot some data on the axes.
    ax.plot(ts_t)
    ax.legend(['Falsa', 'Verdadera'], loc="best")

    plt.xticks(rotation=rotation)

    rotular(ax, 
            title=title,
            xlabel=xlabel,
            ylabel=ylabel,)
    
    if save and save_name:
        save_name = f"{save_name}.pdf"
        save_path = path.join(PATH_IMG, save_name)
        plt.savefig(save_path, bbox_inches="tight")

    plt.show()

### Con respecto a los dias de la semana

In [ ]:
L_dow = ["Monday", "Tuesday", "Wednesday",
         "Thursday", "Friday", "Saturday", "Sunday", ]


def count_dow(ser):
    """Cuenta los días de la semana y los rotúla.
    """
    s = ser.dt.day_name().value_counts()
    s = pd.Series({L_dow[i]: s[L_dow[i]] for i in range(7)})
    return s


ts_t = count_dow(df_true_["date"])
ts_f = count_dow(df_fake_["date"])

my_plot(ts_t, ts_f,
        title="Comparación número de noticias por día de semana",
        xlabel="Día de la semana",
        ylabel="Número de noticias",
        save_name="numero_noticias_dia_semana")

Observamos que las noticias verdaderas sacan noticas de forma constante, mientras que las noticias falsas tiene una baja de más del 50% en los fines de semana.

### Con respecto a los meses

In [ ]:
m = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "Octuber", 11: "November", 12: "December"
}

def number2month(n):
    """Hace un mapeo del número de un mes, al nombre del mes.
    Retorna None si no se entrega un número entero entre el 1 y el 12.
    """
    return m.get(n, None)

def count_month(ser):
    """Cuenta los meses y los rotúla.
    """
    s = ser.dt.month.map(number2month).value_counts()
    s = pd.Series({m[i]:s[m[i]] for i in range(1, 13)})
    return s

ts_f = count_month(df_fake_["date"])
ts_t = count_month(df_true_["date"])

my_plot(ts_t, ts_f, rotation=60,
        title="Comparación número de noticias por mes",
        xlabel="Mes",
        ylabel="Número de noticias",
        save=True,
        save_name="numero_noticias_mes")

Vemos que las noticias verdaderas se mantienen constantes en los meses, mientras que las noticias falsas se mantienen por debajo de las noticias verdaderas. Sin embargo, esta situación cambia en los últimos meses a partir de agosto. Se puede deber que en esos meses empezaron las presidenciales.

### Con respecto a lo largo del tiempo

In [ ]:
ts_f = df_fake_["date"].dropna().apply(
    lambda x: pd.Timestamp(x.strftime("%Y-%m"))
).value_counts().reset_index().sort_values(by=['index'])

ts_t = df_true_["date"].dropna().apply(
    lambda x: pd.Timestamp(x.strftime("%Y-%m"))
).value_counts().reset_index().sort_values(by=['index'])

# Create a figure containing a single axes.
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(ts_f['index'], ts_f['date'], label= 'Falsa')  # Plot some data on the axes.
ax.plot(ts_t['index'], ts_t['date'], label= 'Verdadera')
ax.legend()

rotular(ax,
        title="Noticias por fecha de publicación",
        xlabel="Fecha de publicación",
        ylabel="Frecuencia")

save_name = "noticias_fecha_publicacion.pdf"
save_path = path.join(PATH_IMG, save_name)
plt.savefig(save_path, bbox_inches="tight")

plt.show()

## Nubes de palabras

Se procederá a elaborar las nubes de palabras a partir del cuerpo y el título de las noticias. 

### Preliminares

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
lem = WordNetLemmatizer()
stem = PorterStemmer()
stop_words.add('thi')

In [ ]:
def show_wordcloud(data, title=None,
                   save=SAVE_FIG, save_name=None,):
    """
    Retorna la visualización de una nube de palabras.

    :Param data: str, string del texto que se busca visualizar. 
    :Param title: str, título de la figura generada.
    """

    wordcloud = WordCloud(
        background_color='white',
        stopwords=stop_words,
        max_words=200,
        max_font_size=40,
        scale=3,
        random_state=1  # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title and False:
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)

    if save and save_name:
        save_name = f"{save_name}.pdf"
        save_path = path.join(PATH_IMG, save_name)
        plt.savefig(save_path, bbox_inches="tight")

    plt.show()

### Titulares

In [ ]:
# Ahora para que el formato de entrada a la función que 
# genera las nubes de palabras sea el correcto, debemos unir la lista 
# en un solo string. 

FAKE_TITLE= " ".join(list(df_fake_proc.title))
TRUE_TITLE= " ".join(list(df_true_proc.title))

In [ ]:
show_wordcloud(TRUE_TITLE, 
               title= 'Titulares de noticias verdaderas',
               save_name="wordcloud_titulares_verdaderas")

In [ ]:
show_wordcloud(FAKE_TITLE, 
               title= 'Titulares de noticias falsas',
               save_name="wordcloud_titulares_falsas")

### Cuerpo de la noticia

In [ ]:
# Ahora para que el formato de entrada a la función que genera las nubes de 
# palabras sea el correcto, debemos unir la lista
# en un solo string.

FAKE_TEXT = " ".join(list(df_fake_proc.text))
TRUE_TEXT = " ".join(list(df_true_proc.text))

In [ ]:
show_wordcloud(TRUE_TEXT,
               title='Cuerpo de noticias verdaderas',
               save_name="wordclouds_cuerpo_verdaderas"
               )

In [ ]:
show_wordcloud(FAKE_TEXT,
               title= 'Cuerpo de noticias falsas',
               save_name="wordclouds_cuerpo_falsas"
              )

## Graficos de distribución de frecuencias

In [ ]:
def word_frequency(data, n, col,
                   save=SAVE_FIG, save_name=None,):
    word_array = []
    for row in data[col]:
        for word in row.split(' '):
            word_array.append(word)
    print(f'- Frecuencia por {col}:')
    FreqDist(word_array).plot(n)
    
    if save and save_name:
        save_name = f"{save_name}.pdf"
        save_path = path.join(PATH_IMG, save_name)
        plt.savefig(save_path, bbox_inches="tight")

In [ ]:
print('NOTICIAS VERDADERAS')
word_frequency(df_true_proc, 10, "title",
               save_name="freq_title_true")

word_frequency(df_true_proc, 10, "text",
               save_name="freq_text_true")

In [ ]:
print('NOTICIAS FALSAS')
word_frequency(df_fake_proc, 10, "title",
               save_name="freq_title_fake")

word_frequency(df_fake_proc, 10, "text",
               save_name="freq_text_fake")

In [ ]:
print('TODAS LAS NOTICIAS')
word_frequency(df_proc, 10, "title",
               save_name="freq_title_all")

word_frequency(df_proc, 10, "text",
               save_name="freq_text_all")

# Clasificación

## Selección de Características

Con el texto tokenizado, se formará un conjunto de $k$ variables que representará a cada una de las palabras (distintas) presentes en el conjunto de noticias (cada palabra será una característica). Dicha asociación palabra-variable puede encontrarse en **vectorizer.get_feature_names()** donde el orden de las palabras corresponde a su variable asociada.

Luego, se transformará cada noticia en un vector de $\mathbb{N_0}^k$ donde la $i$-ésima entrada corresponderá a la cantidad de apariciones de la $i$-ésima en la noticia. De este modo, se tiene una única matriz (**encoded_data**) que almacenará cada una de las noticias, lo cual permitirá realizar regresión logística para obtener el clasificador buscado.

Lo único que falta definir es de qué modo se van a entregar los elementos de cada noticia (título y cuerpo). Se proponen las siguientes opciones:

1. Entregar solo los títulos.
2. Entregar solo los cuerpos (contenido).
3. Entregar título y cuerpo como una única cadena (concatenada).

In [ ]:
def data_encoding(data):
    vectorizer = CountVectorizer()
    # matriz de características.
    encoded_data = vectorizer.fit_transform(data).toarray()
    print('Características totales:', len(vectorizer.get_feature_names()))

    print('Características promedio por noticia:')
    print('- Ponderado:',
          np.round(
              np.mean([sum(news_item) for news_item in encoded_data]), 2
          )
         )
    print('- Simple:',
          np.round(
              np.mean(
                  [np.count_nonzero(news_item) for news_item in encoded_data]
              ), 2)
         )
    return encoded_data

In [ ]:
X = dict()

print('\nSOLO TÍTULOS')
X['title'] = data_encoding(df_proc_['title'])

In [ ]:
print('\nSOLO CUERPO')
X['text'] = data_encoding(df_proc_['text'])

In [ ]:
print('\nTITULOS Y CUERPO')
X['title_text'] = data_encoding(
    [news_item for news_item in df_proc_['title'] + ' ' + df_proc_['text']])

## Regresión logística 

In [ ]:
# clasificación real:
y = list(df_proc_['true'])

In [ ]:
columnas_list = ["title", "text", "title & text"]
modelos_list = ["Regresion Logistica", "Naive Bayes", "Perceptron", "SVC"]
resultados = {
    columna: {
        modelo: 0 for modelo in modelos_list
    } for columna in columnas_list
}
resultados

In [ ]:
def classification(x):

    # partición del conjunto:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    print('\nTotal de datos:', len(y))
    print('- Datos de entrenamiento:', len(y_train))
    print('- Datos de validación:', len(y_test))

    # clasificación:
    classifier = LogisticRegression()
    classifier.fit(x_train, y_train)

    def classifier_evaluation(x_eval, y_eval):

        # predicciones y probabilidades de acierto para x_eval:
        y_pred = classifier.predict(x_eval)
        y_prob = classifier.predict_proba(x_eval)*100
        
        precision = np.round(accuracy_score(y_eval, y_pred)*100,2)
        print(f'- Precisión (fracción): {precision}%')
        
        prob_acierto = np.round(np.mean([max(prob) for prob in y_prob]),2)
        print(f'- Probabilidad de acierto media: {prob_acierto}%')
        
        conf_matriz = classification_report(y_eval, y_pred)
        print(f'- Matriz de confusion: \n{conf_matriz}') 

        return precision

    print('\nEvaluación dentro de muestra:')
    classifier_evaluation(x_train, y_train)

    print('\nEvaluación fuera de muestra:')
    precision = classifier_evaluation(x_test, y_test)
    return precision

In [ ]:
print('\nSOLO TÍTULOS')
resultados["title"]["Regresion Logistica"] = classification(X['title'])

In [ ]:
print('\nSOLO CUERPO')
resultados["text"]["Regresion Logistica"] = classification(X['text'])

In [ ]:
print('\nTITULOS Y CUERPO')
resultados["title & text"]["Regresion Logistica"] = classification(X['title_text'])

## Naive bayes

In [ ]:
def classification(x):

    # partición del conjunto:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    print('\nTotal de datos:', len(y))
    print('- Datos de entrenamiento:', len(y_train))
    print('- Datos de validación:', len(y_test))

    # clasificación:
    classifier = GaussianNB()
    classifier.fit(x_train, y_train)

    def classifier_evaluation(x_eval, y_eval):

        # predicciones y probabilidades de acierto para x_eval:
        y_pred = classifier.predict(x_eval)
        y_prob = classifier.predict_proba(x_eval)*100
        
        precision = np.round(accuracy_score(y_eval, y_pred)*100,2)
        print(f'- Precisión (fracción): {precision}%')
        
        prob_acierto = np.round(np.mean([max(prob) for prob in y_prob]),2)
        print(f'- Probabilidad de acierto media: {prob_acierto}%')
        
        conf_matriz = classification_report(y_eval, y_pred)
        print(f'- Matriz de confusion: \n{conf_matriz}') 
        
        return precision

    print('\nEvaluación dentro de muestra:')
    classifier_evaluation(x_train, y_train)

    print('\nEvaluación fuera de muestra:')
    precision = classifier_evaluation(x_test, y_test)
    
    return precision

In [ ]:
print('\nSOLO TÍTULOS')
resultados["title"]["Naive Bayes"] = classification(X['title'])

In [ ]:
print('\nSOLO CUERPO')
resultados["text"]["Naive Bayes"] = classification(X['text'])

In [ ]:
print('\nTITULOS Y CUERPO')
resultados["title & text"]["Naive Bayes"] = classification(X['title_text'])

## Perceptrón

In [ ]:
def classification(x):
    from sklearn.linear_model import Perceptron
    # partición del conjunto:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    print('\nTotal de datos:', len(y))
    print('- Datos de entrenamiento:', len(y_train))
    print('- Datos de validación:', len(y_test))

    # clasificación:
    classifier = Perceptron()
    classifier.fit(x_train, y_train)

    def classifier_evaluation(x_eval, y_eval):

        # predicciones y probabilidades de acierto para x_eval:
        y_pred = classifier.predict(x_eval)
#         y_prob = classifier.predict_proba(x_eval)*100
        
        precision = np.round(accuracy_score(y_eval, y_pred)*100,2)
        print(f'- Precisión (fracción): {precision}%')
        
#         prob_acierto = np.round(np.mean([max(prob) for prob in y_prob]),2)
#         print(f'- Probabilidad de acierto media: {prob_acierto}%')
        
        conf_matriz = classification_report(y_eval, y_pred)
        print(f'- Matriz de confusion: \n{conf_matriz}') 
        
        return precision

    print('\nEvaluación dentro de muestra:')
    classifier_evaluation(x_train, y_train)

    print('\nEvaluación fuera de muestra:')
    precision = classifier_evaluation(x_test, y_test)
    
    return precision

In [ ]:
print('\nSOLO TÍTULOS')
resultados["title"]["Perceptron"] = classification(X['title'])

In [ ]:
print('\nSOLO CUERPO')
resultados["text"]["Perceptron"] = classification(X['text'])

In [ ]:
print('\nTITULOS Y CUERPO')
resultados["title & text"]["Perceptron"] = classification(X['title_text'])

## SVC

In [ ]:
def classification(x):
    from sklearn.svm import LinearSVC
    
    # partición del conjunto:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    print('\nTotal de datos:', len(y))
    print('- Datos de entrenamiento:', len(y_train))
    print('- Datos de validación:', len(y_test))

    # clasificación:
    classifier = LinearSVC()
    classifier.fit(x_train, y_train)

    def classifier_evaluation(x_eval, y_eval):

        # predicciones y probabilidades de acierto para x_eval:
        y_pred = classifier.predict(x_eval)
#         y_prob = classifier.predict_proba(x_eval)*100
        
        precision = np.round(accuracy_score(y_eval, y_pred)*100,2)
        print(f'- Precisión (fracción): {precision}%')
        
#         prob_acierto = np.round(np.mean([max(prob) for prob in y_prob]),2)
#         print(f'- Probabilidad de acierto media: {prob_acierto}%')
        
        conf_matriz = classification_report(y_eval, y_pred)
        print(f'- Matriz de confusion: \n{conf_matriz}') 
        return precision

    print('\nEvaluación dentro de muestra:')
    classifier_evaluation(x_train, y_train)

    print('\nEvaluación fuera de muestra:')
    precision = classifier_evaluation(x_test, y_test)
    
    return precision

In [ ]:
print('\nSOLO TÍTULOS')
resultados["title"]["SVC"] = classification(X['title'])

In [ ]:
print('\nSOLO CUERPO')
resultados["text"]["SVC"] = classification(X['text'])

In [ ]:
print('\nTITULOS Y CUERPO')
resultados["title & text"]["SVC"] = classification(X['title_text'])

## Resultados

In [ ]:
resultados

In [ ]:
res = pd.DataFrame(resultados)
res

In [ ]:
save_name = path.join(PATH_TABLAS, "tabla_resumen.tex")

res.to_latex(save_name,
             column_format="rccc",
             caption="Tabla de Resumen",
             label="tab:resumen",
            )

# Tutorial nltk 

Vale: Ya hice algunos imports y le cambié el nombre a algunas funciones que tenían nombre muy largo. Así que ahora lo voy a explicar: 

tokenizer = RegexpTokenizer(r"\w+") 

Lo que hace es separar un string de un párrafo u oración en una lista de strings con las palabras. La diferencia con la función 'word_tokenize', es que RegexpTokenize puede configurarse para aceptar solamente ciertos caracteres. El argumento r"\w+" es para que ignore la puntuación del texto y no la incluya en la lista. Sé que también se puede configurar para ignorar los números, habría que buscar en google el argumento correcto. 

stem = PorterStemmer()

Lo que hace es devolver una palabra o verbo conjugado a su palabra raíz.

lem = WordNetLemmatizer()

Lo que hace es devolver una palabra o verbo conjugado a su palabra raíz. Dependiendo del segundo argumento que reciba (el primer argumento en el texto como string) puede definir las raices como verbos, pronombres, etc. A diferencia de steamming, relaciona las palabras con su significado, no solamente a partir de la gramática. Esta página (https://www.machinelearningplus.com/nlp/lemmatization-examples-python/) explica que para hacerlo bien hay que usar POS tag, que es otra función que detecta si la palabra es adjetivo, adverbio, sustantivo o verbo, y así WordNetLemmatizer() se puede llamar de forma adecuada para cada palabra.

stop_words=set(stopwords.words("english"))

Es un set con los pronombres, preposiciones, y otro tipo de palabras que no aportan valor. Usando add se pueden incluir palabras nuevas. 

A continuación un ejemplo de uso. 

In [ ]:
txt= df[TITLE][325]

print ( 'El texto original es: ' + txt + '\n')

token_txt= tokenizer.tokenize(txt)

print('El texto toquenizado es: ' )
print(token_txt)
print('\n')

stop_txt=[]
for i in token_txt: 
    if i not in stop_words:
        stop_txt.append(i)

print('El texto sin stop words es: ' )
print(stop_txt)
print('\n')

stem_txt= []
for i in stop_txt:
    stem_txt.append(stem.stem(i))
    
print('El texto stemizado es: ' )
print(stem_txt)  


PD: Cuando estaba programando las nubes de palabras, me di cuenta de que en las noticias falsas, los títulos van siempre con la primera letra de cada palabra en mayúscula (y en las noticias verdaderas no pasa). Podríamos hacer una predicción sin cambiar eso, para ver si considera las mayúsculas como importantes y después otra usando todo en minúscula, porque igual es la media trampa. 